In [ ]:
import os
from pathlib import Path
import boto3

def get_all_s3_objects(s3, **base_kwargs):
    continuation_token = None
    contents = []
    while True:
        list_kwargs = dict(MaxKeys=1000, **base_kwargs)
        if continuation_token:
            list_kwargs['ContinuationToken'] = continuation_token
        response = s3.list_objects_v2(**list_kwargs)
        contents += (response.get('Contents', []))
        if not response.get('IsTruncated'):  # At the end of the list?
            break
        continuation_token = response.get('NextContinuationToken')
    return contents

In [ ]:
s3_client = boto3.client("s3")
bucket_name = "lantern-rd-east2"
slfFiles = get_all_s3_objects(s3_client, Bucket=bucket_name, Prefix="slf")
notSlfFiles = get_all_s3_objects(s3_client, Bucket=bucket_name, Prefix="not-slf")

In [71]:
print(len(slfFiles))
print(len(notSlfFiles))

1339
2812


In [ ]:
for file in slfFiles:
    print(f"File_name: {file['Key']}, size: {file['Size']}")

In [ ]:
fileNum = 0
while os.path.exists("lst-file%s.lst" % fileNum):
    fileNum += 1

output = open("lst-file%s.lst" % fileNum, "w")
    

for i in range(0, len(slfFiles)):
    output.write(str(i) + "\t1\t" + slfFiles[i]['Key'] + '\n')
print("Finished slf: " + str(len(slfFiles)))
for i in range(len(slfFiles), len(notSlfFiles)):
    output.write(str(i) + "\t0\t" + notSlfFiles[i]['Key'] + '\n')
print("Finished not-slf: " + str(len(notSlfFiles)))
output.close()


In [ ]:
missing = 0
for i in range(0, len(slfFiles)):
    match = True
    for j in range(0, len(slfFiles)):
        if i == int(slfFiles[j]['Key'][4:-4]):
            #print(slfFiles[j]['Key'] + " matches with " + str(i))
            match = True
            break
        else: match = False
    if not match:    
        print("Couldn't find " + str(i))
        missing += 1
print("Couldn't find " + str(missing) + " file")
        